# Reto 03
## Jaffet León Chávez

A continuación usted va a desarrollar una red neuronal para detectar el fraude bancario con un dataset reducido del ejemplo 03. Preste atención a los ejercicios y desarrolle cada paso.

In [1]:
# Paso 0. Librerías
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from random import randint
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
import itertools
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [ ]:
# Paso 1. Importe el csv con pandas

In [2]:
df=pd.read_csv('creditcard.csv')

In [ ]:
# Paso 2. 
Genere un EDA, sin visualizaciones para entender las dimensiones de sus variables

In [5]:
df.describe()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,Amount,Class
count,69999.000000,69999.000000,69999.000000,69999.000000,69999.000000,69999.000000,69999.000000,69999.000000,69999.000000,69999.000000,69999.000000
mean,34677.904827,-0.245565,-0.020862,0.681525,0.169179,-0.271138,0.101138,-0.115358,0.057876,97.026195,0.002500
std,14414.367742,1.866626,1.655246,1.419700,1.374950,1.393538,1.307231,1.259253,1.235694,271.173817,0.049938
min,0.000000,-56.407510,-72.715728,-33.680984,-5.172595,-42.147898,-26.160506,-31.764946,-73.216718,0.000000,0.000000
25%,28773.000000,-1.006242,-0.592808,0.194908,-0.724332,-0.889127,-0.636487,-0.604818,-0.141363,7.650000,0.000000
50%,37912.000000,-0.244236,0.071919,0.770956,0.187935,-0.303345,-0.150512,-0.074759,0.066934,26.500000,0.000000
75%,45483.500000,1.153563,0.727725,1.403364,1.052069,0.267790,0.494170,0.419723,0.345569,88.720000,0.000000
max,53665.000000,1.960497,18.902453,4.226108,16.715537,34.801666,22.529298,36.677268,20.007208,19656.530000,1.000000


In [6]:
df.head(45)

,V1,V2,V3,V4,V5,V6,V7,V8,Amount
0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,149.62
1,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,2.69
2,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,378.66
3,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,123.50
4,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,69.99
5,-0.425966,0.960523,1.141109,-0.168252,0.420987,-0.029728,0.476201,0.260314,3.67
6,1.229658,0.141004,0.045371,1.202613,0.191881,0.272708,-0.005159,0.081213,4.99
7,-0.644269,1.417964,1.074380,-0.492199,0.948934,0.428118,1.120631,-3.807864,40.80
8,-0.894286,0.286157,-0.113192,-0.271526,2.669599,3.721818,0.370145,0.851084,93.20
9,-0.338262,1.119593,1.044367,-0.222187,0.499361,-0.246761,0.651583,0.069539,3.68


# Paso 3. 
Filtre solo aquellos casos con fraude (1) para la última variable

In [5]:
y=df.pop('Class')
df.pop('Time')
x=df.values
y=np.array(y)
counter=(np.count_nonzero(y))

print("Datos sobre clientes no fraudulentos " + str(len(y)-counter))
print("Datos sobre clientes fraudulentos " + str(counter))
print("Porcentaje de no fraudulentos " +str(100*(1-(counter/len(y)))))
print("Porcentaje de fraudulentos "+str(100*((counter/len(y)))))

KeyError: 'Class'

In [7]:
x_stand=[]
y_stand=[]
for i in range(len(y)):
    if y[i]==1:
        x_stand.append(x[i])
        y_stand.append(1)
for r in range(2000):
    i=randint(0,len(y)-1)
    if y[i]==0:
        y_stand.append(0)
        x_stand.append(x[i])

x_stand=np.array(x_stand)
y_stand=np.array(y_stand)

## Veamos el equilibrio de mi dataset

In [9]:
print("Datos sobre clientes no fraudulentos " + str(len(y_stand)-counter))
print("Datos sobre clientes fraudulentos " + str(counter))
print("Porcentaje de no fraudulentos " +str(100*(1-(counter/len(y_stand)))))
print("Porcentaje de fraudulentos "+str(100*((counter/len(y_stand)))))

Datos sobre clientes no fraudulentos 1995
Datos sobre clientes fraudulentos 175
Porcentaje de no fraudulentos 91.93548387096774
Porcentaje de fraudulentos 8.064516129032258


# Paso 04. Genere su red neuronal
Tenga cuidado con las layers y nodos que específica. Recuerde: intento y error.
Hagalo definiendo una variable 'modelo' con keras.sequential dentro que con keras.layers.Dense vaya definiendo las capac, una a una, de su respectiva red

In [15]:
epochs=50
model = keras.Sequential([
    keras.layers.Dense(64,input_shape=(9,),activation='relu'),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(64,activation='relu'),
    keras.layers.Dense(2, activation='softmax')
])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 64)                640       
_________________________________________________________________
dense_5 (Dense)              (None, 128)               8320      
_________________________________________________________________
dense_6 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_7 (Dense)              (None, 2)                 130       
Total params: 17,346
Trainable params: 17,346
Non-trainable params: 0
_________________________________________________________________
None


# Paso 05. Compile su modelo con model.compile, y el argumento de optimizer igual a Adam. Juegue con los argumentos posibles.

In [16]:
model.compile(optimizer='Adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

historico=model.fit(x_stand, y_stand, epochs=epochs,validation_split=0.1,verbose=0)

# Genere su Predicción. Bosqueje los resultados e interprete

In [17]:
y = model.predict_classes(x_stand)

for i in range(len(x_stand)):
	print("X=%s, Predicted=%s" % (x_stand[i], y[i]))

X=[-2.31222654  1.95199201 -1.60985073  3.99790559 -0.52218787 -1.42654532
 -2.53738731  1.39165725  0.        ], Predicted=1
X=[-3.04354062e+00 -3.15730712e+00  1.08846278e+00  2.28864362e+00
  1.35980513e+00 -1.06482252e+00  3.25574266e-01 -6.77936530e-02
  5.29000000e+02], Predicted=0
X=[-2.30334957e+00  1.75924746e+00 -3.59744743e-01  2.33024305e+00
 -8.21628328e-01 -7.57875710e-02  5.62319782e-01 -3.99146578e-01
  2.39930000e+02], Predicted=0
X=[-4.39797444  1.35836703 -2.59284422  2.67978697 -1.12813094 -1.70653639
 -3.49619729 -0.24877774 59.        ], Predicted=1
X=[ 1.23423505  3.01974042 -4.30459688  4.73279513  3.62420083 -1.35774566
  1.71344499 -0.49635849  1.        ], Predicted=1
X=[ 0.00843037  4.13783683 -6.24069657  6.67573216  0.76830703 -3.35305955
 -1.63173467  0.15461245  1.        ], Predicted=1
X=[ 0.02677923  4.1324639  -6.56059997  6.34855667  1.32966567 -2.51347885
 -1.6891022   0.3032528   1.        ], Predicted=1
X=[ 0.32959433  3.7128893  -5.77593511  6.07